In [ ]:
# "protobuf<5" "tokenizers>=0.21,<0.22" 是修正 protobuf & tokenizers 版本衝突的語法
# 安裝套件後，會彈出詢問是否【重新啟動工作階段】的訊息，不需理會，此訊息是因為套件會移除 colab 預設的 packages，改用其他版本所引起。
! pip install openai==1.58.1 langchain==0.3.12 langchain_openai==0.2.14 langchain_community==0.3.12 langchain-chroma==0.1.4 "protobuf<5" "tokenizers>=0.21,<0.22"

INFO: pip is looking at multiple versions of chromadb to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of opentelemetry-proto to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of opentelemetry-sdk to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.3/454.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.9/617.9 kB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━

In [ ]:
# 設定 環境變數
import os
from google.colab import userdata


os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_PROJECT'] = 'STORM-Exp'
os.environ['LANGCHAIN_API_KEY'] = userdata.get('LANGCHAIN_API_KEY')

# Document Load

In [ ]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://zh.wikipedia.org/wiki/%E5%8F%B0%E7%81%A3%E5%B0%8F%E5%90%83",),  # 台灣小吃
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("mw-body-content")
        )
    ),
)
docs = loader.load()

# Text Splitter

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

splits = text_splitter.split_documents(docs)


# Create VectorStore & Retriever

In [ ]:
from langchain_chroma import Chroma

vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()


In [ ]:
# 測試 retriever
retriever.invoke('台南小吃有什麼特別之處？')

[Document(metadata={'source': 'https://zh.wikipedia.org/wiki/%E5%8F%B0%E7%81%A3%E5%B0%8F%E5%90%83'}, page_content='各地發展[编辑]\n台北的小吃可分為市集攤販小吃與老店小吃二類。除了賣早點類的小吃攤可能限於早上營業外，夜市小吃集中在夜市裡，由於都市化程度較高，民眾活動時間可能对相較長的關係，許多小吃攤會經營到相當晚，以配合台北人常在夜間活動的性格，因此超過午夜12點甚至徹夜經營也是常有的事情。這類攤販小吃因機動化之故多以免洗餐具為容器。至於歷史較久的老店小吃則集中於發展較早的台北西區一帶，如：艋舺（今萬華區）與建成圓環週圍，和台灣多數發展較早的古城一樣，這些老店多集中於寺廟周圍，又因多有店面方便餐具清潔，故餐具的使用上以非免洗餐具為大宗。 \n\n南臺灣的台南小吃則是代表性的台灣小吃地區，小吃與歷史古蹟搭配的主題旅遊已成該市重要的觀光資源。台南市的著名小吃除了種類眾多物美價廉以外，通常不會集中於夜市，而是在具有歷史發展淵源的廟口廟埕，或是日本统治時期因衛生或都市計畫下遷徙集中的市集（如石精臼或沙卡里巴）這些市集雖可能因為都市發展而消滅或沒落，但四散的攤販為了註明自己的來源出身，有些會在所賣的產品前加註來歷，如「石精臼海產粥」或「大菜市意麵」等。'),
 Document(metadata={'source': 'https://zh.wikipedia.org/wiki/%E5%8F%B0%E7%81%A3%E5%B0%8F%E5%90%83'}, page_content='台灣夜市的炸花枝丸\n擔仔麵\n臺北市松山區新東街的鹹酥雞與香雞排攤位。\n六合夜市盛奕大腸包小腸\n烤玉米\n台灣小吃（臺灣話：小點，臺羅：sió-tiám），是台灣富有文化特色的街頭小吃的總稱[1]，與離島澎湖小吃、馬祖小吃以及金門小吃等共築台灣美食文化的一个重要部分。台灣小吃的特色是就地取材。因為台灣四面環海、漁產豐富，因此海鮮是街頭常見的料理种类之一[2]。\n完全詳盡地定義“台灣小吃”並不容易，習慣上，一般有如下區分：狹義而言，台湾小吃僅指“發源於台灣的小吃”，但由於小吃的變化性極高，易隨地域、時間而突破性地發展，因此廣義上，台湾小吃亦可以包括那些從中國大陸、日本、東南

# 訊息合成

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo")

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough


message = """
請以提供給你的背景資料來回答問題。

問題： {question}

背景資料:
{context}
"""

prompt = ChatPromptTemplate.from_messages([("human", message)])


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
rag_chain.invoke("台南小吃有什麼特色？")

'台南小吃的特色是口味偏甜，這可能是因為台南在日治時期是產糖重鎮，糖很容易取得，因此被用作調味料添加到菜餚中。另外，台南早期居民務農，為了有足夠體力從事農作業，他們在烹調食物時也會加入糖。因此，台南的小吃在口味上偏向甜味。台南小吃通常不會集中在夜市，而是分散在具有歷史發展淵源的廟口廟埕或市集周邊，這些市集可能因城市發展而消失或沒落，但攤販為了註明自己的來源出身，有些會在產品前加註來歷。台南小吃的種類眾多，物美價廉，是台灣小吃的代表性地區之一，也是重要的觀光資源之一。'

# 同學的練習
- 將以上程式碼執行一次，確認可以執行
- 自行挑選一則 wiki 文章來建立 vectorstore
- 實驗【retriever】的結果
- 實驗 rag 的回覆狀況
- 查看 [LangSmith](https://www.langchain.com/langsmith)，觀察執行過程
- 給同學 10 分鐘的時間練習